# Assignment 1

## Importing Libraries and Loading Base Overlay

In [1]:
from pynq.overlays.base import BaseOverlay
import time
from datetime import datetime
base = BaseOverlay("base.bit")

## Inline C/C++ Functions to Interface with PMODB

In [2]:
%%microblaze base.PMODB

#include "gpio.h"
#include "pyprintf.h"

//Function to turn on/off a selected pin of PMODB
unsigned int write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    else {
        gpio pin_out = gpio_open(pin);
        gpio_set_direction(pin_out, GPIO_OUT);
        gpio_write(pin_out, val);
    }
    return 0;
}

//Function to read the value of a selected pin of PMODB
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}

## Testing Blue LED with PMODB Driver

In [ ]:
blue   = 1
green  = 2
red    = 3
period = 0.02
duty_cycle = 0.1
duty_ratio = duty_cycle * period

write_gpio(green, 0)
write_gpio(red,0)

# testing blue, 10 ms 
while(1):
    write_gpio(blue, 1)
    time.sleep(duty_ratio)
    write_gpio(blue, 0)
    time.sleep(period - duty_ratio)

In [ ]:
## Fixed Duty Cycle 25%

In [3]:
import asyncio
import pynq.lib.rgbled as rgbled 
cond = True
# testing loop logic with onboard leds
led4 = rgbled.RGBLED(4)
led5 = rgbled.RGBLED(5)
currentLed = led4
currentVal = 0x1

blue = 1
green = 2
red = 3 

period = 0.01
duty_cycle = 0.25
currentColor = red

async def flash_leds():
    global cond, start, currentLed, currentVal, currentColor, period, duty_cycle
    while True:
        if cond:
            currentLed.write(currentVal)
            write_gpio(currentColor,1)
            await asyncio.sleep(period*duty_cycle)
            write_gpio(currentColor,0)
            currentLed.write(0x0)
            await asyncio.sleep(period-period*duty_cycle)
        else:
            currentLed.write(0x0)
            currentColor.stop()

async def get_btns(_loop):
    global cond, start, currentLed, currentVal, currentColor, red, green, blue
    while cond:
        await asyncio.sleep(0.01)
        if base.buttons[0].read() != 0:
            currentLed.write(0x0)
            currentLed = led4
            currentVal = 0x3
            
            write_gpio(currentColor,0)
            currentColor = red
            write_gpio(currentColor,1)
        if base.buttons[1].read() != 0:
            currentLed.write(0x0)
            currentLed = led4
            currentVal = 0x7
            
            write_gpio(currentColor,0)
            currentColor = green
            write_gpio(currentColor,1)
        if base.buttons[2].read() != 0:
            currentLed.write(0x0)
            currentLed = led5
            currentVal = 0x1
            
            write_gpio(currentColor,0)
            currentColor = blue
            write_gpio(currentColor,1)
        if base.buttons[3].read() != 0:
            _loop.stop()
            cond = False

loop = asyncio.new_event_loop()
loop.create_task(flash_leds())
loop.create_task(get_btns(loop))
loop.run_forever()
loop.close()        
led4.write(0x0)
led5.write(0x0)
write_gpio(green,0)
write_gpio(red,0)
write_gpio(blue,0)
print("Done.")

Done.


## Onboard LED Testing

In [ ]:
import asyncio
import pynq.lib.rgbled as rgbled 
cond = True
# testing loop logic with onboard leds
led4 = rgbled.RGBLED(4)
led5 = rgbled.RGBLED(5)
currentLed = led4
currentVal = 0x1

async def flash_leds():
    global cond, start, currentLed, currentVal
    while True:
        if cond:
            currentLed.write(currentVal)
            await asyncio.sleep(period*duty_cycle)
            currentLed.write(0x0)
            await asyncio.sleep(period-period*duty_cycle)
        else:
            currentLed.write(0x0)

async def get_btns(_loop):
    global cond, start, currentLed, currentVal
    while cond:
        await asyncio.sleep(0.01)
        if base.buttons[0].read() != 0:
            currentLed.write(0x0)
            currentLed = led4
            currentVal = 0x3
        if base.buttons[1].read() != 0:
            currentLed.write(0x0)
            currentLed = led4
            currentVal = 0x7
        if base.buttons[2].read() != 0:
            currentLed.write(0x0)
            currentLed = led5
            currentVal = 0x1
        if base.buttons[3].read() != 0:
            _loop.stop()
            cond = False

loop = asyncio.new_event_loop()
loop.create_task(flash_leds())
loop.create_task(get_btns(loop))
loop.run_forever()
loop.close()        
led4.write(0x0)
led5.write(0x0)
print("Done.")

## Duty Cycle Testing

In [3]:
import asyncio
import pynq.lib.rgbled as rgbled 
cond = True
# testing loop logic with onboard leds
led4 = rgbled.RGBLED(4)
led5 = rgbled.RGBLED(5)
currentLed = led4
currentVal = 0x1

blue = 1
green = 2
red = 3 

period = 0.01
duty_cycle = 0.05
currentColor = red
write_gpio(red,0)
write_gpio(green,0)
write_gpio(blue,0)

async def flash_leds():
    global cond, start, currentLed, currentVal, currentColor, period, duty_cycle
    while True:
        if cond:
            currentLed.write(currentVal)
            write_gpio(currentColor,1)
            await asyncio.sleep(period*duty_cycle)
            write_gpio(currentColor,0)
            currentLed.write(0x0)
            await asyncio.sleep(period-period*duty_cycle)
        else:
            currentLed.write(0x0)
            currentColor.stop()

async def get_btns(_loop):
    global cond, start, currentLed, currentVal, currentColor, red, green, blue, duty_cycle
    while cond:
        await asyncio.sleep(0.1)
        if base.buttons[0].read() != 0:
            currentLed.write(0x0)
            currentLed = led4
            currentVal = 0x3
            
            write_gpio(currentColor,0)
            currentColor = red
            write_gpio(currentColor,1)
        if base.buttons[1].read() != 0:
            currentLed.write(0x0)
            currentLed = led4
            currentVal = 0x7
            
            write_gpio(currentColor,0)
            currentColor = green
            write_gpio(currentColor,1)
        if base.buttons[2].read() != 0:
            currentLed.write(0x0)
            currentLed = led5
            currentVal = 0x1
            
            write_gpio(currentColor,0)
            currentColor = blue
            write_gpio(currentColor,1)
        if base.buttons[3].read() != 0:
            if(duty_cycle < 1):
                duty_cycle = duty_cycle + 0.05
            else:
                _loop.stop()
                cond = False

loop = asyncio.new_event_loop()
loop.create_task(flash_leds())
loop.create_task(get_btns(loop))
loop.run_forever()
loop.close()        
led4.write(0x0)
led5.write(0x0)
write_gpio(green,0)
write_gpio(red,0)
write_gpio(blue,0)
print("Done.")

Done.
